In [18]:
import sys
sys.path.append('/Users/zaneselvans/code/catalyst/pudl')
from pudl import constants, settings, models, ferc1, eia923, pudl

In [20]:
#%load_ext autoreload
%autoreload 1
%aimport pudl
%aimport pudl.ferc1
%aimport pudl.constants
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [24]:
def smallwind(wind_plants,f1_engine):
    small_plant = pd.read_sql('''SELECT p.respondent_id, r.respondent_name, p.report_year, p.plant_name,\
                                p.yr_constructed, p.capacity_rating, p.net_demand, p.net_generation, p.plant_cost,\
                                p.plant_cost_mw, p.operation, p.expns_fuel, p.expns_maint, p.kind_of_fuel,\
                                p.fuel_cost\
                                FROM f1_gnrt_plant p JOIN f1_respondent_id r ON r.respondent_id=p.respondent_id''',f1_engine)

    wind_name = small_plant[small_plant['plant_name'].str.contains("wind", case=False)]
    wind_fuel = small_plant[small_plant['kind_of_fuel'].str.contains("wind", case=False)]
    wind_plants = pd.concat([wind_plants,wind_name,wind_fuel])
    wind_plants.drop_duplicates(inplace=True)
    wind_plants = wind_plants[(wind_plants.yr_constructed!='')]
    return(wind_plants)

In [26]:
ferc1_engine = pudl.ferc1.db_connect_ferc1()
ferc1.init_db(refyear=2015,years=range(2004,2016))
wind_plants = pd.DataFrame()
wind_plants = smallwind(wind_plants, ferc1_engine)

In [28]:
wind_plants.sample(10)

,respondent_id,respondent_name,report_year,plant_name,yr_constructed,capacity_rating,net_demand,net_generation,plant_cost,plant_cost_mw,operation,expns_fuel,expns_maint,kind_of_fuel,fuel_cost
7232,134,PacifiCorp,2014,Glenrock,2008,99.0,100.0,299004000.0,201773148.0,2038113.0,611457.0,0.0,1681532.0,Wind,0.0
6334,161,Southern California Edison Company,2013,SC-ETWIND23-SOL,2010,2.5,2.5,7988085.0,14397052.0,5758821.0,367376.0,0.0,180946.0,,0.0
4539,210,MidAmerican Energy Company,2011,Pomeroy Wind Farm (171 units @ 1.5 MW each &,2007,286.4,55.5,804860850.0,534030686.0,1864632.0,1100470.0,0.0,3806674.0,,0.0
6616,210,MidAmerican Energy Company,2014,Carroll Wind Farm (100 units @ 1.5 MW each),2008,150.0,43.2,517494340.0,302458806.0,2016392.0,619656.0,0.0,2553839.0,,0.0
5410,134,PacifiCorp,2012,Marengo,2007,140.4,139.0,358669000.0,239478535.0,1705688.0,1598918.0,0.0,2137651.0,Wind,0.0
963,80,Kansas Gas and Electric Company,2005,Westar Wind (wind turbines) -- 20%,1999,0.3,0.0,155648.0,415217.0,1384.0,0.0,0.0,3148.0,Wind,0.0
5358,120,Northern States Power Company (Minnesota),2012,Nobles Wind,2010,201.0,203.0,645506722.0,540699166.0,2690046.0,1562757.0,0.0,2057791.0,Wind,0.0
3931,210,MidAmerican Energy Company,2010,Carroll Wind Farm (100 units @ 1.5 MW each),2008,150.0,81.5,430993770.0,300064314.0,2000429.0,647893.0,0.0,1644098.0,,0.0
7637,134,PacifiCorp,2015,Glenrock III,2009,39.0,39.0,108844000.0,87955208.0,2255262.0,210345.0,0.0,517762.0,Wind,0.0
3497,157,Sierra Pacific Power Company,2009,SIERRA PLAZA WIND,2007,0.1,0.0,1.0,107617.0,1076170.0,0.0,0.0,0.0,WIND,0.0


In [50]:

for year in range(2004,2016):
    print('Initializing FERC Form 1 DB for {}'.format(year))
    f1_engine = create_engine('postgresql://catalyst@localhost:5432/ferc1')
    wind_plants = smallwind(wind_plants, f1_engine)

Initializing FERC Form 1 DB for 2004
Initializing FERC Form 1 DB for 2005
Initializing FERC Form 1 DB for 2006
Initializing FERC Form 1 DB for 2007
Initializing FERC Form 1 DB for 2008
Initializing FERC Form 1 DB for 2009
Initializing FERC Form 1 DB for 2010
Initializing FERC Form 1 DB for 2011
Initializing FERC Form 1 DB for 2012
Initializing FERC Form 1 DB for 2013
Initializing FERC Form 1 DB for 2014
Initializing FERC Form 1 DB for 2015


In [54]:
wind_plants.to_csv('ferc1_wind_plants_2004-2015.csv')

In [19]:
pudl_engine = create_engine('postgresql://catalyst@localhost:5432/pudl_sandbox')

In [13]:
f1_fuel = pd.read_sql('''SELECT respondent_id, report_year, plant_name, fuel, fuel_unit, fuel_quantity,\
                                fuel_avg_heat, fuel_cost_delvd, fuel_cost_burned, fuel_cost_btu, fuel_cost_kwh,\
                                fuel_generaton \
                                FROM f1_fuel WHERE plant_name <> '' AND fuel NOT IN ('Total','') ''',f1_engine)

In [14]:
f1_fuel.columns

Index(['respondent_id', 'report_year', 'plant_name', 'fuel', 'fuel_unit',
       'fuel_quantity', 'fuel_avg_heat', 'fuel_cost_delvd', 'fuel_cost_burned',
       'fuel_cost_btu', 'fuel_cost_kwh', 'fuel_generaton'],
      dtype='object')

In [18]:
f1_fuel.sample(10)

,respondent_id,report_year,plant_name,fuel,fuel_unit,fuel_quantity,fuel_avg_heat,fuel_cost_delvd,fuel_cost_burned,fuel_cost_btu,fuel_cost_kwh,fuel_generaton
908,195,2015,Total - Pulliam,Oil,Barrels,0.0,0.0,0.000,0.000,0.000,0.000,0.000
181,294,2015,Holland,NG,mmBtu,1932634.0,1.0,3.010,3.010,3.010,0.025,8401.000
145,186,2015,Possum Point,Gas,mcf,486102.0,1058.0,5.090,5.450,5.150,0.069,0.000
450,145,2015,Fruita,Oil,bbls,146.0,136765.0,117.680,117.680,20.500,0.000,0.000
436,145,2015,Hayden,Coal,tons,704321.0,11658.0,45.030,48.440,2.130,0.025,11841.590
1407,144,2015,GALLAGHER,Oil,Barrels,21594.0,137000.0,80.201,86.709,15.069,0.004,0.000
1238,130,2015,MUSTANG,Gas,MCF,1295042.0,1037000.0,0.000,35.958,0.033,0.396,11832.000
1115,51,2015,Asbury,Tires,Tons,2098.0,13936.0,35.949,35.949,1.290,0.014,10502.686
782,56,2015,Turkey Point 5,Oil,Barrels,11890.0,137476.0,60.517,60.517,10.481,0.000,0.000
893,195,2015,Pulliam 5,Oil,Barrels,0.0,0.0,0.000,0.000,0.000,0.000,0.000


In [ ]:
pis = { 1: {301:'intangible_org', 302:'intangibal_franch'}}